# LISTA 1

## GLOBAL SETUP

In [31]:
import numpy as np
import os
import statistics
from tqdm import tqdm
import re
import nltk
import collections
from tqdm import tqdm
import random
import itertools

## ZADANIE 1

In [33]:
def Succ1Grams():
    data = []
    with open("1grams", "r") as f:
        line = f.readline()
        while line:
            t = line.split(" ")[-1].replace("\n", "")
            if len(t) < 50:
                data.append(t)
            line = f.readline()
            if len(data) > 1000000:
                break
    return data

singletons = Succ1Grams()
maxlen = max(len(s) for s in singletons)
print(f"read {len(singletons)}, longest one is {maxlen}")
maxlen = 25

read 1000001, longest one is 49


In [3]:
def OwnMatch(w):
    graph = [[] for _ in w]
    for slit in range(len(w)):
        for forward_succ in range(5):
            taken = w[slit:slit+forward_succ+1]
            if len(taken) == forward_succ+1:
                if taken in singletons:
                    graph[slit].append(slit + forward_succ + 1)

    #print(f"neighbor matrix")
    #display(graph)

    dp = [(0, None) for _ in range(len(w)+1)]
    for i in range(len(w)-1):
        for jt in graph[i]:
            if dp[jt][0] < (dp[i][0] + (i-jt)**2):
                dp[jt] = (dp[i][0] + (i-jt)**2, i)

    #display(dp)

    baked = []
    ls = dp[-1][1]
    fs = len(w)
    while ls != None:
        baked.append(w[ls:fs])
        fs = ls
        ls = dp[ls][1]
        
    baked.reverse()
    
    return baked 

In [4]:
def MaxMatch(w, verbose = False):
    tokenized = []
    succ = 1
    best = (0, "reeee")
    
    while len(w) > 0:
        try_ = w[:succ]
        if try_ in singletons:
            if verbose:
                print(f"found candidate {try_}")
            best = (succ, try_) 

        succ += 1
        
        if succ > maxlen or try_ == w:
            if best[0] == 0:
                raise Exception(f"could not match {w[:succ]}")
            else:
                if verbose:
                    print(f"longest match was {best[1]}")
                tokenized.append(best[1])
                if verbose:
                    print(f"transforming {w} -> {w[best[0]:]}")
                w = w[best[0]:]
                succ = 1
                best = (0, "reeee")
    return tokenized

In [6]:
# implementation stolen from https://blog.paperspace.com/implementing-levenshtein-distance-word-autocomplete-autocorrect/
# since jupyter considered that importing
# python-levenstein is on the same level as breaking the geneva
# convention

def levenshteinDistanceDP(token1, token2):
    distances = np.zeros((len(token1) + 1, len(token2) + 1))

    for t1 in range(len(token1) + 1):
        distances[t1][0] = t1

    for t2 in range(len(token2) + 1):
        distances[0][t2] = t2
        
    a = 0
    b = 0
    c = 0
    
    for t1 in range(1, len(token1) + 1):
        for t2 in range(1, len(token2) + 1):
            if (token1[t1-1] == token2[t2-1]):
                distances[t1][t2] = distances[t1 - 1][t2 - 1]
            else:
                a = distances[t1][t2 - 1]
                b = distances[t1 - 1][t2]
                c = distances[t1 - 1][t2 - 1]
                
                if (a <= b and a <= c):
                    distances[t1][t2] = a + 1
                elif (b <= a and b <= c):
                    distances[t1][t2] = b + 1
                else:
                    distances[t1][t2] = c + 1

    return distances[len(token1)][len(token2)]

In [8]:
def perform_test(compare_string):
    digested = compare_string.replace(" ", "")
    p1 = " ".join(MaxMatch(digested))
    p2 = " ".join(OwnMatch(digested))
    s1 = levenshteinDistanceDP(compare_string, p1)
    s2 = levenshteinDistanceDP(compare_string, p2)
    #print(compare_string)
    #print(f"MaxMatch returned {p1}, score {s1}")
    #print(f"OwnCrap returned {p2}, score {s2}")
    return (s1, s2)

In [34]:
poem = '''Jedzie Wojtuś popod lasem, skrajem dąbrowy,
Wiezie se do domu laser, laser gazowy.
Pytali się go kolesie, po co to nabył?
- A bo właśnie stał w GS-ie, więc kupiłem go Teresie.
Dyć coś zawsze przywieźć chce się dla swej baby!
Jedzie Wojtuś popod lasem, po twardym dukcie.
Wiezie se do domu laser, czyta instrukcję.
A instrukcja jest ciekawa: Włączyć do prądu,
To ten laser wszystko skrawa, bez różnicy, stal czy trawa
I w ogóle jest zabawa prawie bez swądu.
Jedzie Wojtuś popod lasem, woła: Wio, wiśta!
Wiezie se do domu laser, tak se rozmyśla:
"Jak w tym dojdę do biegłości, będzie wygodnie,
Jest tu paru wrednych gości, przyceluję se w skrytości
I padalcom z odległości podpalę spodnie!"
Jedzie Wojtuś popod lasem, rad, że o rany
Wiezie se do domu laser, układa plany:
"Niech spróbuje na rowerze jeździć Kaczmarski,
Zaraz w dętkę mu przymierzę, i kartofle się obierze,
I wywierci dziury w serze, by był śwajcarski!"
Jedzie Wojtuś popod lasem, skręcił przy POM-ie.
Wiezie se do domu laser schowany w słomie.
Na podwórku cielę bryka, kaczki na stawie...
Wdziera, wdziera się technika coraz częściej w dom rolnika,
Jeszcze to nie Ameryka.... Ale już prawie!!! '''

tMM, tMC = [], []
for l in tqdm(poem.split("\n")):
    retMM, retMC = perform_test(l.lower().replace(",", ""))
    tMM.append(retMM)
    tMC.append(retMC)
    
print("MaxMatch:")
print(f"Mean: {statistics.mean(tMM)}")
print(f"Stdev: {statistics.stdev(tMM)}")
print(f"max: {max(tMM)}")
print(f"min: {min(tMM)}")

print("MyCrap:")
print(f"Mean: {statistics.mean(tMC)}")
print(f"Stdev: {statistics.stdev(tMC)}")
print(f"max: {max(tMC)}")
print(f"min: {min(tMC)}")

#MaxMatch:
#Mean: 5.96
#Stdev: 4.6141087980237305
#max: 17.0
#min: 0.0
#MyCrap:
#Mean: 13.68
#Stdev: 12.294036494712927
#max: 48.0
#min: 4.0

100%|██████████| 25/25 [01:48<00:00,  4.32s/it]

MaxMatch:
Mean: 6.2
Stdev: 3.9370039370059056
max: 13.0
min: 0.0
MyCrap:
Mean: 7.4
Stdev: 2.0
max: 11.0
min: 2.0


## ZADANIE 2 i 3


In [9]:
def calculate_bigrams():
    bigrams = []
    with open("2grams", "r") as f:
        line = f.readline()
        while line:
            line = re.sub(' +', ' ', line)
            toks = line.replace("\n", "").split(" ")
            bigrams.append((int(toks[1]), toks[2], toks[3]))
            line = f.readline()
            if len(bigrams) > 10000000:
                break
    return bigrams
            
bigrams = calculate_bigrams()

In [10]:
for b in bigrams[:10]:
    print(b)
for b in bigrams[-10:]:
    print(b)


(311157, 'się', 'w')
(252085, 'w', 'tym')
(216704, 'się', 'na')
(203946, 'się', 'z')
(187137, 'się', 'do')
(160694, 'nie', 'ma')
(142720, 'nie', 'jest')
(137760, 'się,', 'że')
(121382, 'w', 'tej')
(116683, 'jest', 'to')
(2, 'cpu', '(central')
(2, 'cpu', 'fsb')
(2, 'cpu.', 'im')
(2, 'cpu.', 'jądro')
(2, 'cpu', 'lub')
(2, 'cpu', 'podczas')
(2, 'cpu.', 'procesory')
(2, 'cpu.', 'przylutowane')
(2, 'cpu', 'ratio')
(2, 'cpu', 'składa')


In [11]:
bigram_dict = collections.defaultdict(lambda: [])
for count, first, second in tqdm(bigrams):
    bigram_dict[first].append((count, second))

100%|██████████| 10000001/10000001 [00:08<00:00, 1184198.88it/s]


In [12]:
def generate_story(max_len, starting, distribution):
    if max_len == 0:
        max_len = 1000000
    if starting == None:
        starting = random.choice(list(bigram_dict.keys()))
        
    while(max_len > 0):
        print(starting, end = ' ')
        max_len -= 1
        candidates = bigram_dict[starting]
        
        if distribution == "random":
            candidates = [c[1] for c in candidates]
            if len(candidates) == 0:
                return
            starting = random.choice(candidates) 
            
        if distribution == "weight":
            if len(candidates) == 0:
                return
            all_ents = sum(c[0] for c in candidates)
            chosen = random.randint(0, all_ents)
            for it in range(len(candidates)):
                chosen -= candidates[it][0]
                if chosen <= 0:
                    starting = candidates[it][1]
                    break
                    
            

In [37]:
for i in range(10):
    generate_story(0, "papież", "random")
    print("\n=======")

papież wysiadł pan niezgodnie ze wcześniejszym zakończeniu sprawy, problemy społeczności ludzi pociąga kilka ofiar. a tematy zawodowe. zresztą ówczesna nie dostaną, bo centrala, nie zarzucać, że biura, z przeklętych i gwarantowanych przez palaczy — gwałtowny obrót w rojewie. 
papież gotowy i wąchać klej się pianą. - patryk! 
papież pisze badacz przeszłości miało miejsce... – zdecydowali rodzice. o wszystkim”, 
papież mimo siwych koni, gdy chiński pokój, jak niepewność. choć nasze kontry już podpisują kontroler 8237a, przechowująca dokumenty (m.in. właściciel pałacu, założonego w oecd. w interesy usa dla skrajnego indywidualizmu, który przedstawiliśmy, jest inwestowane. 
papież ucałował ją. nic zrazu wydawało. w midraszach 
papież nas wreszcie my. nas trzymał ich wydania, 4) sędziego rodzinnego jest substytutem dla „solidarności”, ale technologicznie i łazienka czy uciec, zaszyć - zaproponowano jego odbiciem. to patrzę wprost przez ostatnie wyceny działek przeciwlotniczych w snuciu plan

## ZADANIE 4

In [39]:
def permutation_analizer(sentence):
    results = []
    for p in itertools.permutations(sentence):
        score = 1.0
        for k in range(len(p) - 1):
            candidates = bigram_dict[p[k]]
            if candidates == []:
                score -= 100.0
            else:
                commit = False
                for c in candidates:
                    if c[1] == p[k+1]:
                        score += c[0]
                        commit = True
                if not commit:
                    score -= 100.0
                    
        results.append((-score, p))
    
    res = sorted(results)
    for r in res:
        print(r)
            
permutation_analizer("wczoraj wieczorem spotkałem pewną piękną kobietę".split(" "))
#permutation_analizer(["judyta", "dała", "wczoraj", "stefanowi", "czekoladki"])

(-806.0, ('wczoraj', 'wieczorem', 'spotkałem', 'pewną', 'piękną', 'kobietę'))
(-789.0, ('wczoraj', 'wieczorem', 'spotkałem', 'pewną', 'kobietę', 'piękną'))
(-701.0, ('pewną', 'piękną', 'kobietę', 'wczoraj', 'wieczorem', 'spotkałem'))
(-701.0, ('piękną', 'kobietę', 'wczoraj', 'wieczorem', 'spotkałem', 'pewną'))
(-695.0, ('pewną', 'piękną', 'kobietę', 'spotkałem', 'wczoraj', 'wieczorem'))
(-695.0, ('spotkałem', 'pewną', 'piękną', 'kobietę', 'wczoraj', 'wieczorem'))
(-695.0, ('spotkałem', 'wczoraj', 'wieczorem', 'pewną', 'piękną', 'kobietę'))
(-684.0, ('pewną', 'kobietę', 'piękną', 'wczoraj', 'wieczorem', 'spotkałem'))
(-684.0, ('piękną', 'wczoraj', 'wieczorem', 'spotkałem', 'pewną', 'kobietę'))
(-678.0, ('kobietę', 'piękną', 'wczoraj', 'wieczorem', 'spotkałem', 'pewną'))
(-678.0, ('kobietę', 'wczoraj', 'wieczorem', 'spotkałem', 'pewną', 'piękną'))
(-678.0, ('pewną', 'kobietę', 'piękną', 'spotkałem', 'wczoraj', 'wieczorem'))
(-678.0, ('spotkałem', 'pewną', 'kobietę', 'piękną', 'wczoraj', 